<h1>Acromelic Frontonasal Dysostosis (ZSWIM6): Smith et al 2014</h1>
<p>Extract the clinical data from <a href="https://pubmed.ncbi.nlm.nih.gov/25105228/"target="__blank">Smith JD, et al. (2014) Exome sequencing identifies a recurrent de novo ZSWIM6 mutation associated with acromelic frontonasal dysostosis. Am J Hum Genet ;95(2):235-40 PMID: 25105228</a>.<p>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import CohortValidator
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.9


In [2]:
pmid="PMID:25105228"
title="Exome sequencing identifies a recurrent de novo ZSWIM6 mutation associated with acromelic frontonasal dysostosis"
cite = Citation(pmid=pmid, title=title)
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_csv("input/smith_2014.csv", delimiter="\t")

In [4]:
df.head()

,Feature,Individual 1,Individual 2,Individual 3,Individual 4
0,Periventricular nodular heterotopia,no,yes,no,no
1,Ventriculomegaly,yes,no,no,yes
2,Choroid plexus cyst,yes,yes,no,no
3,Dilation of Virchow-Robin spaces,no,yes,no,no
4,Agenesis of corpus callosum,no,yes,no,no


<h2>Phenotypic data</h2>

In [5]:
from collections import defaultdict
patient_d = defaultdict(list)

In [6]:
for _, row in df.iterrows():
    label = row["Feature"]
    hpo_term = hpo_cr.get_term_from_label(label)
    individual_1 = row["Individual 1"]
    individual_2 = row["Individual 2"]
    individual_3 = row["Individual 3"]
    individual_4 = row["Individual 4"]
    if individual_1 == "yes":
        hpt1 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label)
    elif individual_1 == "no":
        hpt1 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label, observed=False)
    patient_d["individual_1"].append(hpt1)
    if individual_2 == "yes":
        hpt2 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label)
    elif individual_2 == "no":
        hpt2 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label, observed=False)
    patient_d["individual_2"].append(hpt2)
    if individual_3 == "yes":
        hpt3 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label)
    elif individual_3 == "no":
        hpt3 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label, observed=False)
    patient_d["individual_3"].append(hpt3)
    if individual_4 == "yes":
        hpt4 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label)
    elif individual_4 == "no":
        hpt4 = HpTerm(hpo_id=hpo_term.id, label=hpo_term.label, observed=False)
    patient_d["individual_4"].append(hpt4)
    #@print(f"{label}: {hpo_term}")

<h2>Variant</h2>
<p>Sequencing of ZSWIM6 (RefSeq NM_020928.1)  confirms the exon 14 c.3487C>T mutation that leads to the amino acid substitution p.Arg1163Trp in all four cases.</p>

In [7]:
validator = VariantValidator(genome_build='hg38')
transcript = "NM_020928.2"
var_hgvs = "c.3487C>T"
v = validator.encode_hgvs(hgvs=var_hgvs, custom_transcript=transcript)
v.set_heterozygous()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_020928.2%3Ac.3487C>T/NM_020928.2?content-type=application%2Fjson


In [8]:
individual_list = []
disease_label = "Acromelic frontonasal dysostosis"
disease_id = "OMIM:603671"
dysostosis = Disease(disease_id=disease_id, disease_label=disease_label)
for pat_id, hpo_list in patient_d.items():
    if hpo_list is None:
        print(f"Could not find hpo list for {pat_id}")
        continue
    if len(hpo_list) == 0:
        print(f"warning, empty HPO list for {pat_id}")
    ind = Individual(individual_id=pat_id, 
                     hpo_terms=hpo_list, 
                     interpretation_list=[v.to_ga4gh_variant_interpretation()], 
                     disease=dysostosis)
    individual_list.append(ind)
    
print(f"Created {len(individual_list)} individual objects")

Created 4 individual objects


In [9]:
cvalidator = CohortValidator(cohort=individual_list, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,2


In [10]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
individual_1 (UNKNOWN; ),Acromelic frontonasal dysostosis (OMIM:603671),NM_020928.2:c.3487C>T (heterozygous),Telecanthus (HP:0000506); Hypopituitarism (HP:0040075); Calcification of falx cerebri (HP:0005462); Patellar hypoplasia (HP:0003065); Choroid plexus cyst (HP:0002190); Midline central nervous system lipomas (HP:0006866); Hypertelorism (HP:0000316); Preaxial foot polydactyly (HP:0001841); Ptosis (HP:0000508); Large sella turcica (HP:0002690); Parietal foramina (HP:0002697); Short tibia (HP:0005736); Aplasia of the olfactory bulb (HP:0032466); Ventriculomegaly (HP:0002119); Hypoplasia of the corpus callosum (HP:0002079); Vertical clivus (HP:0010559); Cryptorchidism (HP:0000028); Bifid nose (HP:0011803); Midline facial cleft (HP:0100629); Glaucoma (HP:0000501); Global developmental delay (HP:0001263); Cleft palate (HP:0000175); excluded: Periventricular nodular heterotopia (HP:0032388); excluded: Dilation of Virchow-Robin spaces (HP:0012520); excluded: Agenesis of corpus callosum (HP:0001274); excluded: Absent septum pellucidum (HP:0001331); excluded: Retrocerebellar cyst (HP:0006951); excluded: Myopia (HP:0000545); excluded: Optic nerve hypoplasia (HP:0000609); excluded: Remnants of the hyaloid vascular system (HP:0007968); excluded: Submucous cleft soft palate (HP:0011819)
individual_2 (UNKNOWN; ),Acromelic frontonasal dysostosis (OMIM:603671),NM_020928.2:c.3487C>T (heterozygous),Telecanthus (HP:0000506); Hypopituitarism (HP:0040075); Calcification of falx cerebri (HP:0005462); Patellar hypoplasia (HP:0003065); Choroid plexus cyst (HP:0002190); Myopia (HP:0000545); Midline central nervous system lipomas (HP:0006866); Hypertelorism (HP:0000316); Agenesis of corpus callosum (HP:0001274); Preaxial foot polydactyly (HP:0001841); Ptosis (HP:0000508); Parietal foramina (HP:0002697); Dilation of Virchow-Robin spaces (HP:0012520); Short tibia (HP:0005736); Aplasia of the olfactory bulb (HP:0032466); Submucous cleft soft palate (HP:0011819); Vertical clivus (HP:0010559); Remnants of the hyaloid vascular system (HP:0007968); Bifid nose (HP:0011803); Midline facial cleft (HP:0100629); Periventricular nodular heterotopia (HP:0032388); Global developmental delay (HP:0001263); excluded: Ventriculomegaly (HP:0002119); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Absent septum pellucidum (HP:0001331); excluded: Large sella turcica (HP:0002690); excluded: Retrocerebellar cyst (HP:0006951); excluded: Glaucoma (HP:0000501); excluded: Optic nerve hypoplasia (HP:0000609)
individual_3 (UNKNOWN; ),Acromelic frontonasal dysostosis (OMIM:603671),NM_020928.2:c.3487C>T (heterozygous),Hypertelorism (HP:0000316); Telecanthus (HP:0000506); Bifid nose (HP:0011803); Short tibia (HP:0005736); Global developmental delay (HP:0001263); Midline facial cleft (HP:0100629); Calcification of falx cerebri (HP:0005462); Patellar hypoplasia (HP:0003065); Preaxial foot polydactyly (HP:0001841); Myopia (HP:0000545); Absent septum pellucidum (HP:0001331); Ptosis (HP:0000508); Vertical clivus (HP:0010559); Large sella turcica (HP:0002690); Optic nerve hypoplasia (HP:0000609); Midline central nervous system lipomas (HP:0006866); Parietal foramina (HP:0002697); excluded: Periventricular nodular heterotopia (HP:0032388); excluded: Ventriculomegaly (HP:0002119); excluded: Choroid plexus cyst (HP:0002190); excluded: Dilation of Virchow-Robin spaces (HP:0012520); excluded: Agenesis of corpus callosum (HP:0001274); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Retrocerebellar cyst (HP:0006951); excluded: Glaucoma (HP:0000501); excluded: Remnants of the hyaloid vascular system (HP:0007968); excluded: Submucous cleft soft palate (HP:0011819); excluded: Hypopituitarism (HP:0040075); excluded: Cryptorchidism (HP:0000028)
individual_4 (UNKNOWN; ),Acromelic frontonasal dysostosis (OMIM:603671),NM_020928.2:c.3487C>T (heterozygous),Hypertelorism (HP:0000316); Telecanthus (HP:0000506); Bifid 

In [11]:
Individual.output_individuals_as_phenopackets(individual_list=individual_list,
                                              metadata=metadata)

We output 4 GA4GH phenopackets to the directory phenopackets
